IACS summer camp - OpenCV
===================

[OpenCV](#openCV) is a python module that provides python bindings to a set of fast and efficient computer vision libraries written in C and C++. These bindings mean we can write code in python to access all of the openCV tools  without having to write any C or C++, while we still get the benefits of high sped processing of the compiled code.
In this session we'll be introducing some of the tools available in the python module, and developing some code to automatically count seals visible in high resolution satellite imagery.

###Installing openCV
Windows
Run the installation exe file and remember where you extracted the openCV files too.
When the installer has finished go to the opencv/build/python/2.7 folder. Find the file called `cv2.pyd` and copy it the `/lib/site-packeges` subfolder of your python directory. eg. `C:/Python27/lib/site-packeges`.

Mac

`conda install -c https://conda.binstar.org/jjhelmus opencv`

https://samkhan13.wordpress.com/2012/06/18/using-opencv-with-python-on-your-mac-os-x/

Homebrew install: ```ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)"```

1. ```brew tap homebrew/science```
2. ```brew install opencv```
3. ```sudo su```
4. ```echo "/usr/local/lib/python2.7/site-packages/" > /Library/Python/2.7/site-packages/opencv.pth```

###Importing modules

You'll need to load several modules that we'll use in our seal detector. First we load the openCV package `cv2`. openCV images are stored as array of numbers, with rows and columns of the array representing the rows and columns of the image, with each value being the intensity or colour of a pixel.  openCV uses another module, `numpy`, to handle the arrays. We will import it ``as np``, which essentially just renames the module from `numpy` to `np` in our code. openCV needs `numpy` and assumes it has been renamed `np`, so this step is important.

In [1]:
import cv2
import numpy as np

### Loading an image
Loading an image is simple in openCV, we use the `imread` function. The first argument (`file_name`) is the location of the image we want to load. It should be a string surrounded by quotation marks. The second argument (`flag`) tells openCV how we want to read the file.

```python
img = cv2.imread(<file_name>,<flag>)
```

> **Flags:**

> - **cv2.IMREAD_COLOR** : Loads the image in color, but ignores the alpha (transparency) channel if present.
> - **cv2.IMREAD_GRAYSCALE** : Loads the image in grayscale. 
> - **cv2.IMREAD_UNCHANGED** : Loads the image as it is, with alpha channel included.

If you don't specify a flag then by default the image is loaded in its original form.

When specifying a filename make sure you include the file extension. openCV will use this to work out what format the image is in and load it appropriately.


**If you enter an invalid file name openCV won't tell you that there's a problem and the image created will be empty. Make sure you get the right path!**

> Open CV supports reading:
> 
 - .bmp
 - .jpg
 - .tif
 - .png
>
 >plus a few more

In [2]:
img = cv2.imread('testSeals.jpg',cv2.IMREAD_COLOR)

The image is stored as array called img. We can access the value of any pixel using its array index if we know its row and column position. If the image is in grayscale, the pixel has a single value, its intensity. If the image is in a color format such as rgb the pixel value is a tuple containing a value for the levels of red, green and blue hat make up the color at that pixel.

In [3]:
#Get value of pixel at 100,100
img[100,100]
#Get value of pixel at 100,100 from blue channel
img[100,100,1]

242

###Displaying an image
OpenCV provides functions for viewing the images we've loaded, but they can be a little confusing. Initially we need to create a window in which we'll display the image, and choose the image we want to display.
```python
cv2.imshow(<'window_name'>,<image>)
```
We use the `imshow` function with the first argument (`window_name`) being the name of the window (a string) and the second argument (`image`) being the image we want to display.



In [4]:
cv2.imshow('Seals',img)

If you run the code up to this point you'll see that an empty window is created, but our image doesn't show up yet. We need to tell python to draw the image and wait until we're done with it.  `waitKey()` tells openCV to draw the image an wait until we press any key before doing anything else.  The argument to this function is a time (in milliseconds) to wait for a key press before continuing anyway. 0 means wait forever.
```python
cv2.waitKey(<time>)
```

In [5]:
cv2.waitKey(0)

32

When we're done with the image we can close the window using the `destroyAllWindows()` function. This line (and any that follow) won't be run until either a key has been pressed or `<time>` milliseconds have passed.

In [6]:
cv2.destroyAllWindows()

####Complete code example - displaying an image

In [2]:
#Load the image
img = cv2.imread('simpleBlobs.jpg',cv2.IMREAD_COLOR)
#Create a window to display the image
cv2.imshow('seal_image',img)
#Draw the image and wait for user input
cv2.waitKey(0)
#When a key has ben pressed, destroy the window
cv2.destroyAllWindows()

###Saving an image
```python
cv2.imwrite('file_name',image)
```
To save an image we use the `imwrite` function. The first argument is the path to save the file as, including the extension (.jpg, .png, etc.), and the second argument is the image you'd like to save. If the just give the function the name of the file to save it will be saved in the directory you run python from.

####Code example - Saving a copy of the seal image

In [3]:
cv2.imwrite('Seal_copy.png',img)

True


##Blob Detection
###The theory of blob detection

**explain blob detection steps here**

----------

----------

###cv2.SimpleBlobDetector()
The openCV function `cv2.SimpleBlobDetector()` implements a simple algorithm that performs all the necessary steps to detect blobs in an image.



----------

```python
detector = cv2.SimpleBlobDetector()
```

cv2.SimpleBlobDetector() takes a single argument which is a parameters object. We'll cover this in the next section, so for now you can leave it empty and the function will use the default parameters which are set to detect dark circular objects in images.
The function returns a *detector* object. The reasons for this extra step are complicated and we wont get into them here. Essentially it means that we can use a wide variety of feature detection methods, but we make sure they all have the same methods and attributes so that you don't have to rewrite all your code if you decide to change the method you use. 


----------

```python
detector.detect(image)
```

The detector object we create `detector` has a method `.detect()` that takes one argument, an image to run detection on. The function returns a list with one *keypoint* object for each blob detected. A *keypoint* is openCVs way of storing the results of feature detectors, and it contains the position of each feature in the image.

The simpleBlobDetector detector expects to be given an 8-bit greyscale image. 8-bit refers to the way the number is stored by the computer. Computers work in binary expressing numbers in base 2 as a series of 1's or 0's. An 8-bit number is represented by 8 bits (1's or 0's), meaning that there are a maximum of 2^8 (256) numbers we can store. This includes 0, so an 8-bit number can take a value from 0 to 255. This value represents the intensity of the grey scale image at a pixel; 0 is black and 255 is white.
We can make sure the image is an 8-bit greyscale image by setting the right flag when we load the image. Alterantively openCV allows us to convert images between formats, using the function `cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)`.

----------



`cv2.drawKeypoints(<image>,<keypoints>, <np.array([])>, <color>, <flags>)`

We can mark the image with the detected blobs using the `cv2.drawKeypoints()` function. It takes 5 arguments... `image` is an image on which to overlay the detected blobs. `keypoints` is the list of detected blobs returned from `detector.detect()`. `np.array([])` is an extra output image we can overlay features on with the right flags. We will be putting the point on the input image so we don't need an output image, but openCV expects to get a numpy array here, so we pass it an empty array using `np.array([])`.  `color` sets the color of the markers added to the image in rgb format. `flags` takes a flag object that determins how the markers are drawn. The options we are interested in are:
> **Flags:**

> - **cv2.DRAW_MATCHES_FLAGS_DEFAULT** : Draws center points of blobs over image.
> - **cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS** : For each blob draws a circle with area proportional to area of detected blob.

###Challenge  - Detecting simple circular blobs
Use the skills you've learned so far to write a function to detect the circles in the simple_blobs.png image.  The input to the function should be the file path, it should display the image with the detected blobs, wait for the user to press a key,  and then return the number of circles in the image.
If you have time you can try using the [`cv2.putText()`](http://docs.opencv.org/modules/core/doc/drawing_functions.html) to overlay the count on the image.

####Code Solution - Detecting simple circular blobs

In [4]:
def detectBlobs(path):
    
    #Load the image
    img = cv2.imread(path)
    
    # Set up the detector with default parameters.
    detector = cv2.SimpleBlobDetector()

    # Detect blobs.
    blobs = detector.detect(img)

    #Overlay detected blobs on the original image
    imBlobs = cv2.drawKeypoints(img,blobs, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    numBlobs = len(blobs)
    
    #select font for text overlay
    font = cv2.FONT_HERSHEY_SIMPLEX

    #add text to image
    cv2.putText(imBlobs,str(numBlobs),(10,500), font, 4,(100,0,0),2)

    # Show image with keypoints
    cv2.imshow("Blobs", imBlobs)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return numBlobs



In [5]:
detectBlobs('simpleBlobs.jpg')

22

----------
Unfortunately Seals aren't usually perfectly circular dots in satellite images. Try running your blob detecting function on the `simpleseals.png` image and see what happens.

It probably told you there were no seals in the image. This is because with the default parameters `SimpleBlobDetector()` is detecting all the blobs in the image, but then filtering them on the basis of color and shape and only returning those that correspond to black circles. We can change this  default behavior by changing the argument `params` when we first create the detector object.
The argument should be a simpleBlobDetector parameters object. You can create this object like this 
```python
params = cv2.SimpleBlobDetector_Params()
```

The `params` object has an attribute for each of the avialable filters. You can get an idea about what these are by running `dir(params)` which will print the methods and attributes associated with the params object.
Some of the attributes are boolean (True or False) and turn filters on or off, while the others set the values associated with the filters.
When you change the filters from the defaults you get a whole new set of defaults that might give you weird results if you're not careful. Its best to make sure you explicitly switch off any filters you dont intend to use.

### Challenge - Detecting seals
Modify your function from challenge 1 by turning off the filters and try to count the seals in the `simpleSeals.png` image.

In [6]:
def detectBlobs(path):
    
    #Load the image
    img = cv2.imread(path)
    
    #Create the parameters object
    params = cv2.SimpleBlobDetector_Params()
    
    #Set params attributes to turn filters off
    params.filterByColor = False
    params.filterByArea = False
    params.filterByCircularity = False
    params.filterByConvexity = False
    params.filterByInertia = False
    
    # Set up the detector with default parameters.
    detector = cv2.SimpleBlobDetector(params)

    # Detect blobs.
    blobs = detector.detect(img)

    #Overlay detected blobs on the original image
    imBlobs = cv2.drawKeypoints(img,blobs, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    numBlobs = len(blobs)
    
    #select font for text overlay
    font = cv2.FONT_HERSHEY_SIMPLEX

    #add text to image
    cv2.putText(imBlobs,str(numBlobs),(10,500), font, 4,(100,0,0),2)

    # Show image with keypoints
    cv2.imshow("Blobs", imBlobs)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return numBlobs

Turning off all the filters isn't really a great solution. If there are any other blob like regions in the image we're going to find those too. Try running your function on the `mixedblobs.png` image and see what happens. 
### Challenge - Detecting only seals
Modify your function to detect only the seals in the `mixedblobs.png` image. If you have time try modifying it so that you can seperatley count both the seals in the image and the number of non-seal blobs.

In [8]:
detectBlobs('mixedBlobs.jpg')

16

In [31]:
def detectBlobs(img):
    
    #Load the image
    #img = cv2.imread(path)
    
    #Create the parameters object
    params = cv2.SimpleBlobDetector_Params()
    
    #Set params attributes to turn filters off
    params.filterByColor = False
    params.filterByArea = True
    params.filterByConvexity = False
    params.filterByInertia = False
    
    # Filter by Circularity
    params.filterByCircularity = True
    params.minCircularity = 0
    params.maxCircularity = 0.5
    
    # Filter by Area
    params.minArea = 1
    
    # Set up the detector with default parameters.
    detector = cv2.SimpleBlobDetector(params)

    # Detect blobs.
    blobs = detector.detect(img)

    #Overlay detected blobs on the original image
    imBlobs = cv2.drawKeypoints(img,blobs, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    numBlobs = len(blobs)
    
    #select font for text overlay
    font = cv2.FONT_HERSHEY_SIMPLEX

    #add text to image
    cv2.putText(imBlobs,str(numBlobs),(10,500), font, 4,(100,0,0),2)

    # Show image with keypoints
    cv2.imshow("Blobs", imBlobs)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return numBlobs


In [13]:
detectBlobs('mixedBlobs.jpg')

4

In [16]:
img = cv2.imread('mixedBlobs.jpg',cv2.IMREAD_COLOR)

In [21]:
cv2.imshow('Blob',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
def enlarge_image(image):
    '''Adds a 1 pixel border around each image to help with contour detection.'''
    
    size = len(image)
    
    new image = np.zeros[(size,size,3)]
    
    # Add 1 to R
    for i=1:size+2:
        for j = 1:size+2:
            
            


In [37]:
replicate = cv2.copyMakeBorder(img,100,100,100,100,cv2.BORDER_CONSTANT,value=(255,255,255))

In [38]:
detectBlobs(replicate)

7